step1:
将原来6G的xml文件切分成平均1M的小文档

In [3]:
RUN = False
if(RUN):  
    fin = open('zhwiki-20171020-pages-articles-multistream.xml',encoding='utf8')
    for i in range(10000000):      
        readin = fin.readlines(100*1000)
        if(not readin):
            break
        fout = open('parts/part_'+str(i),'w',encoding = 'utf8')
        fout.writelines(readin)
        # fout.writelines(re.sub("[\[\]\'\"\@#$%^&*()<>：{}|/_.+-—，,。!！ ？、~@#￥%……&*（）]+", "",s))
        fout.close()
    fin.close()
    partsNum = i
    print(partsNum)

49125


判断一个unicode是否为汉字

In [3]:
def is_ch(uchar):
        """判断一个unicode是否是汉字"""
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False

导入utf8编码表，构造一个dic，将汉字映射到整数

In [4]:
with open('GBK',encoding='utf8') as fin:
    chList = []
    for line in fin.readlines():
        for word in line:
            if(is_ch(word)):
                chList.append(word)

                
chNum = len(chList)
print(chNum)
ch2int = {}
idx = 0
with open('ch2int','w',encoding='utf8') as fout:  
    for w in chList:
        ch2int[w] = idx
        fout.write(w+'\t'+str(idx)+'\n')
        idx += 1


20902


定义类BitSet 用于集合运算

In [5]:
class BitSet:
    def __init__(self, elementNum):
        self.bytesNum = round(elementNum/8)
        self.set = bytearray(self.bytesNum)
        
    def insert(self, index):
        if(index >= 0 and index>>3 < self.bytesNum):
            self.set[index >> 3] = self.set[index >> 3] | (1 << int(index & 7)) 
            return True;
        return False;
    
    def isElement(self, index):
        if(index >= 0 and index>>3 < size and (array[index >> 3] & (1<<(index & 7)))):
            return True
        return False
        
    def AND(self, bs):
        a = BitSet(self.bytesNum*8)
        for i in range(self.bytesNum):
            a.set[i] = self.set[i] & bs.set[i]
        return a

给每一个汉字创建倒排索引，对应包含该汉字的文档

In [6]:
partsNum = 49125
bytesNum = round(partsNum/8)

invertedIndex = [BitSet(partsNum) for i in range(chNum)]
print(invertedIndex[0].bytesNum)

6141


显示进度条

In [7]:
import sys,time

def processBar(index, totalNum):
    bar = '#'*int((index/totalNum)*50)
    sys.stdout.write(str(int((index/totalNum)*100))+'%  ||'+bar+'->'+"\r")
    #sys.stdout.flush()

遍历所有文档的所有字符，建立倒排索引

In [21]:

#for partIdx in range(partsNum):
for partIdx in range(100):   
    processBar(partIdx, partsNum)
    
    with open('parts/part_'+str(partIdx),encoding='utf8') as f:
        for word in f.read():
            try:
                if(is_ch(word)):
                    invertedIndex[ch2int[word]].insert(partIdx)
            except KeyError as err:
                print('KEY ERROR'+str(err))

for chIdx in range(chNum):    
    processBar(chIdx, chNum)
    with open('iidx/ch_'+str(chIdx)+'.iidx','w',encoding='utf8') as f:
            for j in invertedIndex[chIdx].set:
                f.write(str(j)+' ')

测试：读取倒排索引

In [8]:
def readIidx(bSet, file):
    with open(file,encoding='utf8') as f:
        s = f.read()[:-1].split(' ')
    assert(len(s) == b.bytesNum)
    for i in range(bSet.bytesNum):
        bSet.set[i] = int(s[i])
    
b = BitSet(partsNum)
f = 'iidx/ch_0.iidx'
readIidx(b, f)
print(b.set[:20])

bytearray(b'\x07\x08\x06\x00\x002\n\x0c\x00\x18\x08\x14\x06\x00\x00\x00\x00\x00\x00\x00')


建立每个文档的 tf 表

In [ ]:
# tf 
RUN_tf = False

if(RUN_tf):
    for partIdx in range(partsNum):

        tf = [0.0 for i in range(chNum)]
        total = 0

        with open('parts/part_'+str(partIdx),encoding='utf8') as f:
            for word in f.read():
                if(is_ch(word)):
                    tf[ch2int[word]] += 1
                    total += 1

        total = max(total, 1)
        for i in range(chNum):
            tf[i] /= total

        with open('parts/tf/part_'+str(partIdx)+'.tf','w',encoding='utf8') as fout:
            for i in tf:
                fout.write(str(i)+' ')
                
        processBar(partIdx, partsNum)


计算每个单词的 idf 值

In [26]:
# idf(ch) = log(total DocumentNum / Num of documents containing ch)

idf = [0.0 for i in range(chNum)]

for chIdx in range(chNum):
    num = 0
    iidxFile = 'iidx/ch_'+str(chIdx)+'.iidx'
    bs = BitSet(partsNum)
    readIidx(bs, iidxFile)
    for i in range(partsNum):
        if(bs.isElement(i)):
            num += 1
    idf[chIdx] = log(partsNum/num)

2